In [146]:
import pandas as pd
import numpy as np
import datetime as dt
import quant_risk_mgmt as qrm
from scipy.stats import t, norm, kurtosis, skew
from scipy.optimize import fsolve, minimize
import inspect
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats

## Problem 1

Using the data in “problem1.csv”

a. Calculate Log Returns (2pts)

b. Calculate Pairwise Covariance (4pt)

c. Is this Matrix PSD? If not, fix it with the “near_psd” method (2pt) 

d. Discuss when you might see data like this in the real world. (2pt)

## a. Calculate Log Returns (2pts)

In [177]:
problem1 = pd.read_csv('problem1.csv')

In [148]:
problem1return = qrm.return_calculate(problem1, method="LOG").drop('Date', axis=1)
problem1return

,Price1,Price2,Price3
1,-0.000167,0.006431,0.004410
2,-0.003615,0.001817,0.014347
3,0.004779,-0.017533,-0.013907
4,-0.002998,-0.001988,-0.004369
5,-0.000108,0.005709,-0.005682
6,0.004314,0.018403,0.036238
7,-0.001676,-0.003510,-0.020447
8,0.001221,-0.000076,0.016141
9,-0.004328,-0.020475,-0.020122
10,0.005081,NaN,NaN


## b. Calculate Pairwise Covariance (4pt)

In [149]:
def missing_cov(x, fun=np.cov):
    n, m = x.shape
    nMiss = np.sum(np.isnan(x), axis=0)

    idxMissing = [set(np.where(np.isnan(x[:, i]))[0]) for i in range(m)]
        
    out = np.empty((m, m))
    for i in range(m):
        for j in range(i+1):
            rows = set(range(n))
            for c in (i,j):
                for rm in idxMissing[c]:
                    if rm in rows:
                        rows.remove(rm)
            rows = sorted(list(rows))
            out[i,j] = fun(x[rows,:][:,[i,j]].T)[0,1]
            if i != j:
                out[j,i] = out[i,j]
    return out

In [150]:
pairwise_covariance = missing_cov(problem1return.values)
pd.DataFrame(pairwise_covariance)

,0,1,2
0,0.000011,0.000018,0.000019
1,0.000018,0.000241,0.000150
2,0.000019,0.000150,0.000331


## c. Is this Matrix PSD? If not, fix it with the “near_psd” method (2pt) 

In [151]:
qrm.is_psd(pairwise_covariance)

True

## d. Discuss when you might see data like this in the real world. (2pt)

Missing values are very common in finance. Not all markets are open at the same time on the same days. A holiday in one market is not necessarily a holiday in another, even in the same country.


## problem 2. 

“problem2.csv” contains data about a call option. Time to maturity is given in days. Assume 255 days in a year.

a. Calculate the call price (1pt)

b. Calculate Delta (1pt)

c. Calculate Gamma (1pt)

d. Calculate Vega (1pt)

e. Calculate Rho (1pt)

Assume you are long 1 share of underlying and are short 1 call option. Using Monte Carlo assuming a Normal distribution of arithmetic returns where the implied volatility is the annual volatility and 0 mean

f. Calculate VaR at 5% (2pt)

g. Calculate ES at 5% (2pt)

h. This portfolio’s payoff structure most closely resembles what? (1pt)

In [152]:
problem2 = pd.read_csv('problem2.csv')
problem2

,Underlying,Strike,IV,TTM,RF,DivRate
0,88.853335,87.787841,0.18,148,0.045,0.056976


In [153]:
S0 = problem2["Underlying"][0]
K = problem2["Strike"][0]
sigma = problem2["IV"][0]
r = problem2["RF"][0]
q = problem2["DivRate"][0]
T1 = problem2["TTM"][0]/255


In [154]:
class option:
    def __init__(self, S, K, T, r, q, sigma, option_type):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.q = q
        self.sigma = sigma
        self.option_type = option_type
        self.d1 = (np.log(self.S / self.K) + (self.r - self.q + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))
        self.d2 = self.d1 - self.sigma * np.sqrt(self.T)
    
    def price_euro(self):
        if self.option_type == 'call':
            price = self.S * np.exp(-self.q * self.T) * norm.cdf(self.d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(self.d2)
        elif self.option_type == 'put':
            price = self.K * np.exp(-self.r * self.T) * norm.cdf(-self.d2) - self.S * np.exp(-self.q * self.T) * norm.cdf(-self.d1)
        
        return price
    
    def delta_closed(self):
        if self.option_type == 'call':
            delta = np.exp(-self.q * self.T) * norm.cdf(self.d1)
        elif self.option_type == 'put':
            delta = -np.exp(-self.q * self.T) * norm.cdf(-self.d1)
        
        return delta
    
    def gamma_closed(self):
        gamma = np.exp(-self.q * self.T) * norm.pdf(self.d1) / (self.S * self.sigma * np.sqrt(self.T))
        
        return gamma
    
    def vega_closed(self):
        vega = self.S * np.exp(-self.q * self.T) * norm.pdf(self.d1) * np.sqrt(self.T)
        
        return vega
    
    def theta_closed(self):
        if self.option_type == 'call':
            theta = -self.S * np.exp(-self.q * self.T) * norm.pdf(self.d1) * self.sigma / (2 * np.sqrt(self.T)) - self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(self.d2) + self.q * self.S * np.exp(-self.q * self.T) * norm.cdf(self.d1)
        elif self.option_type == 'put':
            theta = -self.S * np.exp(-self.q * self.T) * norm.pdf(self.d1) * self.sigma / (2 * np.sqrt(self.T)) + self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(-self.d2) - self.q * self.S * np.exp(-self.q * self.T) * norm.cdf(-self.d1)
        
        return theta
    
    def rho_closed(self):
        if self.option_type == 'call':
            rho = self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(self.d2)
        elif self.option_type == 'put':
            rho = -self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(-self.d2)

        return rho


    def carry_rho_closed(self):
        if self.option_type == 'call':
            carry_rho =  self.S * self.T * np.exp(-self.q * self.T) * norm.cdf(self.d1)
        elif self.option_type == 'put':
            carry_rho =  - self.S * self.T * np.exp(-self.q * self.T) * norm.cdf(-self.d1)

        return carry_rho

    
    def greeks_closed(self):
        value = self.price_euro()
        delta = self.delta_closed()
        gamma = self.gamma_closed()
        vega = self.vega_closed()
        theta = self.theta_closed()
        rho = self.rho_closed()
        carry_rho = self.carry_rho_closed()
        
        return {'price': value, 'Delta': delta, 'Gamma': gamma, 'Vega': vega, 'Rho': rho}

In [155]:
option1 = option(S0, K, T1, r, q, sigma, 'call')
print(option1.greeks_closed())

{'price': 4.909593025658097, 'Delta': 0.5245155199819024, 'Gamma': 0.03149985250406671, 'Vega': 25.98065062194632, 'Rho': 24.199659907128797}


## week7 Problem2

## Assume you are long 1 share of underlying and are short 1 call option. Using Monte Carlo assuming a Normal distribution of arithmetic returns where the implied volatility is the annual volatility and 0 mean

f. Calculate VaR at 5% (2pt)

g. Calculate ES at 5% (2pt)

h. This portfolio’s payoff structure most closely resembles what? (1pt)

In [158]:
def var(data, mean=0, alpha=0.05):
  return mean - np.quantile(data, alpha)

def ES(a, alpha=0.05):
    x = np.sort(a)
    nup = int(np.ceil(a.size * alpha))
    ndn = int(np.floor(a.size * alpha))
    v = 0.5 * (x[nup] + x[ndn])
    
    es = np.mean(x[x <= v])
    return -es

In [156]:
simulated_returns = np.random.normal(0, sigma, (255, 1000))
simulated_returns

array([[-0.08708355,  0.23185027, -0.02337817, ...,  0.27744805,
        -0.1586409 , -0.07989033],
       [ 0.19120334,  0.17152656,  0.11255766, ...,  0.07976812,
        -0.11386342,  0.14322856],
       [ 0.2875401 ,  0.04309256, -0.12815427, ..., -0.0484494 ,
        -0.27230478, -0.25944263],
       ...,
       [ 0.26670055, -0.02708298,  0.28352002, ...,  0.48675586,
         0.09029997, -0.09870216],
       [-0.07766084, -0.25247963, -0.27896088, ...,  0.18349464,
         0.0377827 ,  0.03714204],
       [-0.14882659,  0.18060321, -0.00819595, ...,  0.27566829,
        -0.38762401,  0.18784421]])

In [157]:
simulate_prices_stock = S0 * np.exp(simulated_returns.cumsum(axis=0))
simulate_prices_stock

array([[8.14430120e+01, 1.12037917e+02, 8.68001992e+01, ...,
        1.17264860e+02, 7.58187926e+01, 8.20309612e+01],
       [9.86035118e+01, 1.33001966e+02, 9.71412962e+01, ...,
        1.27002054e+02, 6.76591613e+01, 9.46631982e+01],
       [1.31452685e+02, 1.38858645e+02, 8.54569134e+01, ...,
        1.20995560e+02, 5.15307122e+01, 7.30309029e+01],
       ...,
       [1.32556432e+00, 6.83693640e+00, 7.73421124e+03, ...,
        4.48054252e+03, 2.70275015e+02, 1.06969501e+03],
       [1.22651576e+00, 5.31142469e+00, 5.85146833e+03, ...,
        5.38296196e+03, 2.80682102e+02, 1.11017272e+03],
       [1.05691135e+00, 6.36276677e+00, 5.80370597e+03, ...,
        7.09157217e+03, 1.90489812e+02, 1.33958493e+03]])

In [159]:
#Var = var(simulate_prices)
#ES = ES(simulate_prices)

## h. This portfolio’s payoff structure most closely resembles what? (1pt)

This is resemble short a Put.

## Problem 3.

Data in “problem3_cov.csv” is the covariance for 3 assets. “problem3_ER.csv” is the expected return for each asset as well as the risk free rate.

a. Calculate the Maximum Sharpe Ratio Portfolio (4pt)

b. Calculate the Risk Parity Portfolio (4pt)

c. Compare the differences between the portfolio and explain why. (2pt)

In [160]:
cov_matrix = pd.read_csv('problem3_cov.csv')
cov_matrix

,Asset1,Asset2,Asset3
0,0.031992,0.013559,0.017372
1,0.013559,0.028380,0.016362
2,0.017372,0.016362,0.046582


In [161]:
expected_return = pd.read_csv('problem3_ER.csv')
expected_return

,RF,Expected_Value_1,Expected_Value_2,Expected_Value_3
0,0.045,0.119261,0.114944,0.13461


In [162]:
assets = ['1','2','3']
exp_return = np.array([0.119261, 0.114944, 0.13461])
rf = 0.045

In [163]:
def max_sr(stocks, stockMeans, covar, rf, bound=(0,None)):

    # Define Sharpe ratio function
    def sr(w):
        m = np.dot(w, stockMeans) - rf
        s = np.sqrt(np.dot(w.T, np.dot(covar, w)))
        return m / s

    n = len(stocks)

    # Define optimization problem
    bounds = [bound] * n
    cons = {"type": "eq", "fun": lambda w: np.sum(w) - 1}
    result = minimize(lambda w: -sr(w), np.ones(n) / n, method="SLSQP", bounds=bounds, constraints=cons)

    # Extract optimal weights and other information
    w = result.x
    w = w / np.sum(w)
    Weights = pd.DataFrame({"Stock": stocks, "Weight": w, "cEr": stockMeans * w})

    return Weights, sr(w)

In [164]:
weights, sharp_ratio = max_sr(assets, exp_return,cov_matrix, rf, (0,None))
weights

,Stock,Weight,cEr
0,1,0.346084,0.041274
1,2,0.367058,0.042191
2,3,0.286858,0.038614


## b. Calculate the Risk Parity Portfolio (4pt)

In [165]:
def risk_parity(stockMeans, covar, b=None):
    n = len(stockMeans)
    
    # Function for Portfolio Volatility
    def pvol(w):
        x = np.array(w)
        return np.sqrt(x.dot(covar).dot(x))
    
    # Function for Component Standard Deviation
    def pCSD(w, b=None, last = False):
        x = np.array(w)
        pVol = pvol(w)
        csd = x * (covar.dot(x)) / pVol
        if last:
            return csd
        if b is not None:
            csd /= b
        return csd
    
    # Sum Square Error of cSD
    def sseCSD(w):
        csd = pCSD(w, b)
        mCSD = np.sum(csd) / n
        dCsd = csd - mCSD
        se = dCsd * dCsd
        return 1.0e5 * np.sum(se) # Add a large multiplier for better convergence
    
    # Define the optimization problem
    m = minimize(sseCSD, [1/n]*n, method='SLSQP', bounds=[(0, None)]*n, constraints={'type': 'eq', 'fun': lambda w: np.sum(w)-1})
    
    w = m.x
    
    # Compute RPWeights
    RPWeights = pd.DataFrame({
        'Weight': w,
        'cEr': stockMeans * w,
        'CSD': pCSD(w, b, True)
    })
    
    return RPWeights

In [166]:
risk_parity(exp_return, cov_matrix)

,Weight,cEr,CSD
0,0.345964,0.041260,0.049245
1,0.367326,0.042222,0.049246
2,0.286710,0.038594,0.049246


## c. Compare the differences between the portfolio and explain why. (2pt)

The weights are similar between the portfolios.  That is the result we have when we constrain weights to be positive.
 

## Problem 4

Data in “problem4_returns.csv” is a series of returns for 3 assets. “problem4_startWeight.csv” is the starting weights of a portfolio of these assets as of the first day in the return series.

a. Calculate the new weights for the start of each time period (2pt)

b. Calculate the ex-post return attribution of the portfolio on each asset (4pt)

c. Calculate the ex-post risk attribution of the portfolio on each asset (2pt)

## a. Calculate the new weights for the start of each time period (2pt)

In [167]:
problem4_returns = pd.read_csv('problem4_returns.csv').drop('Date', axis=1)
problem4_returns

,Asset1,Asset2,Asset3
0,0.123235,-0.059703,-0.011278
1,0.032524,0.013558,-0.013804
2,-0.028624,-0.023935,-0.075374
3,0.001476,-0.056679,0.031728
4,-0.032290,-0.108642,-0.064155
5,-0.027250,0.023838,0.028650
6,-0.021056,-0.033035,-0.057953
7,0.029539,0.022435,0.046869
8,-0.033067,-0.018433,-0.020233
9,0.000511,-0.012726,0.031626


In [168]:
problem4_startWeight = pd.read_csv('problem4_startWeight.csv')
problem4_startWeight

,weight1,weight2,weight3
0,0.249956,0.37158,0.378463


## b. Calculate the ex-post return attribution of the portfolio on each asset (4pt)

## c. Calculate the ex-post risk attribution of the portfolio on each asset (2pt)

In [169]:
stocks = ['Asset1',  'Asset2' , 'Asset3']

In [170]:
def Ex_Post_attribution(w, Returns):
    n = Returns.shape[0]
    pReturn = np.empty(n)
    weights = np.empty((n, len(w)))
    lastW = np.copy(w)
    matReturns = Returns[stocks].values

    for i in range(n):
        # Save Current Weights in Matrix
        weights[i,:] = lastW

        # Update Weights by return
        lastW = lastW * (1.0 + matReturns[i,:])

        # Portfolio return is the sum of the updated weights
        pR = np.sum(lastW)
        # Normalize the wieghts back so sum = 1
        lastW = lastW / pR
        # Store the return
        pReturn[i] = pR - 1

    # Set the portfolio return in the Update Return DataFrame
    Returns['Portfolio'] = pReturn

    # Calculate the total return
    totalRet = np.exp(np.sum(np.log(pReturn + 1))) - 1
    # Calculate the Carino K
    k = np.log(totalRet + 1) / totalRet

    # Carino k_t is the ratio scaled by 1/K 
    carinoK = np.log(1.0 + pReturn) / pReturn / k
    # Calculate the return attribution
    attrib = pd.DataFrame(matReturns * (weights * carinoK[:, np.newaxis]), columns=stocks)

    # Set up a Dataframe for output.
    Attribution = pd.DataFrame({'Value': ["TotalReturn", "Return Attribution"]})


    for s in stocks + ['Portfolio']:
        # Total Stock return over the period
        tr = np.exp(np.sum(np.log(Returns[s] + 1))) - 1
        # Attribution Return (total portfolio return if we are updating the portfolio column)
        atr =  attrib[s].sum() if s != 'Portfolio' else tr
        # Set the values
        Attribution[s] = [tr, atr]

        # Y is our stock returns scaled by their weight at each time
        Y =  matReturns * weights
        # Set up X with the Portfolio Return
        X = np.column_stack((np.ones((pReturn.shape[0], 1)), pReturn))
        # Calculate the Beta and discard the intercept
        B = (np.linalg.inv(X.T @ X) @ X.T @ Y)[1,:]
        # Component SD is Beta times the standard Deviation of the portfolio
        cSD = B * np.std(pReturn)

        Expost_Attribution = pd.concat([Attribution,    
            pd.DataFrame({"Value": ["Vol Attribution"], 
                        **{stocks[i]: [cSD[i]] for i in range(len(stocks))},
                        "Portfolio": [np.std(pReturn)]})
        ], ignore_index=True)

    return Expost_Attribution
Ex_Post_attribution(problem4_startWeight.values[0], problem4_returns)

,Value,Asset1,Asset2,Asset3,Portfolio
0,TotalReturn,-0.180664,-0.445798,0.139049,-0.158183
1,Return Attribution,-0.047821,-0.164726,0.054364,-0.158183
2,Vol Attribution,0.005706,0.007313,0.016062,0.029081


## Problem 5

Input prices in “problem5.csv” are for a portfolio. You hold 1 share of each asset. Using arithmetic returns, fit a generalized T distribution to each asset return series. Using a Gaussian Copula:

a. Calculate VaR (5%) for each asset (3pt)

b. Calculate VaR (5%) for a portfolio of Asset 1 &2 and a portfolio of Asset 3&4 (4pt)

c. Calculate VaR (5%) for a portfolio of all 4 assets. (3pt)

In [171]:
problem5 = pd.read_csv('problem5.csv')

In [178]:
return5 = qrm.return_calculate(problem5).drop('Date', axis=1)
return5
newest_prices = problem5.drop('Date',axis=1).tail(1).values[0]


In [173]:
def MLE_t(params, returns):
    df, loc, scale = params
    neg_LL = -1 * np.sum(stats.t.logpdf(returns, df=df, loc=loc, scale=scale))
    return(neg_LL)

def fit_t(returns):
    constraints=({"type":"ineq", "fun":lambda x: x[0]-1}, 
                 {"type":"ineq", "fun":lambda x: x[2]})
    
    returns_t = minimize(MLE_t, x0=[10, np.mean(returns), np.std(returns)], args=returns, constraints=constraints)
    df, loc, scale = returns_t.x[0], returns_t.x[1], returns_t.x[2]
    return df, loc, scale

In [174]:
def gaussian_copula(returns, fitting_model=None, n_sample=10000, seed=0):
    stocks = returns.columns.tolist()
    n = len(stocks)

    if fitting_model is None:
        fitting_model = np.full(n, 't')


    # Fitting model for each stock
    parameters = []
    assets_returns_cdf = pd.DataFrame()
    for i, stock in enumerate(stocks):
        if fitting_model[i] == 't':
            params = fit_t(returns[stock])
            fitting = 't'
        elif fitting_model[i] == 'n':
            params = norm.fit(returns[stock])
            fitting = 'n'
        parameters.append(params)
        assets_returns_cdf[stock] = t.cdf(returns[stock],df=params[0], loc=params[1], scale = params[2]) if fitting == 't' else norm.cdf(returns[stock],loc=params[0], scale = params[1])

    # Simulate N samples with spearman correlation matrix
    np.random.seed(seed)
    spearman_corr_matrix = assets_returns_cdf.corr(method='spearman')
    sim_sample = qrm.multivar_norm_simu(spearman_corr_matrix, method='pca')
    sim_sample = pd.DataFrame(sim_sample, columns=stocks)

    # Convert simulation result with cdf of standard normal distribution
    sim_sample_cdf = pd.DataFrame()
    for stock in stocks:
        sim_sample_cdf[stock] = norm.cdf(sim_sample[stock],loc=0,scale=1)
            
    # Convert cdf matrix to return matrix with parameter
    sim_returns = pd.DataFrame()
    for i, stock in enumerate(stocks):
        if fitting_model[i] == 't':       
            sim_returns[stock] = t.ppf(sim_sample_cdf[stock], df=parameters[i][0], loc=parameters[i][1], scale = parameters[i][2])
        elif fitting_model[i] == 'n':
            sim_returns[stock] = norm.ppf(sim_sample_cdf[stock],  loc=parameters[i][0], scale = parameters[i][1])
    
    return sim_returns, pd.DataFrame(parameters,index=[stocks,fitting_model])

In [176]:
return5

,Price1,Price2,Price3,Price4
1,0.000196,-0.000265,0.000207,-0.000396
2,0.000231,0.000307,-0.000420,0.000495
3,-0.000054,-0.000461,0.000279,-0.000742
4,0.000006,0.000394,0.000356,0.000835
5,0.000127,-0.000054,-0.000146,-0.000468
6,0.000024,-0.000051,-0.000204,0.000511
7,-0.000799,0.000319,-0.000649,-0.000667
8,0.000800,-0.000015,0.000745,0.000223
9,-0.000313,-0.000078,-0.000048,-0.000088
10,0.000256,-0.000090,0.000015,0.000164


In [179]:
sim_returns, parameters = gaussian_copula(return5)
sim_returns

,Price1,Price2,Price3,Price4
0,-1.688889e+06,-817924.752797,-1.088583e+06,-0.000511
1,-8.828649e+05,-813253.157875,-1.240747e+06,-0.000033
2,-1.149544e+06,-816756.492094,-8.156304e+04,-0.000728
3,-2.362004e+06,-816374.686961,-7.344290e+05,-0.000984
4,-1.447444e+06,-842714.458165,-3.457337e+05,-0.000822
...,...,...,...,...
24995,-1.687326e+06,-814208.015699,-4.051577e+05,-0.000453
24996,5.107973e+05,-816590.870597,-1.181628e+06,-0.000493
24997,-5.602798e+05,-815787.517601,-2.487074e+05,0.000318
24998,-1.345305e+05,-811569.166520,5.814320e+05,0.000295


## a. Calculate VaR (5%) for each asset (3pt)

In [181]:
for i, asset in enumerate(sim_returns.columns):
    single_VaR = var(sim_returns[asset])
    single_dollar_VaR = var(sim_returns[asset] * newest_prices[i])

    print("VaR is {}, $VaR is {}.".format(i+1,single_VaR, single_dollar_VaR))

VaR is 1, $VaR is 1855026.708050733.
VaR is 2, $VaR is 826147.863828112.
VaR is 3, $VaR is 1356411.7770189291.
VaR is 4, $VaR is 0.0006696211116501918.


## b. Calculate VaR (5%) for a portfolio of Asset 1 &2 and a portfolio of Asset 3&4 (4pt)

In [182]:
VaR_1_2 = var(sim_returns[['Price1','Price2']].dot(newest_prices[:2]))
print("VaR is {:.6f}, $VaR is {:.6f}.".format(VaR_1_2/sum(newest_prices[:2]),VaR_1_2))

VaR is 1348244.639797, $VaR is 284775967.944942.


In [185]:
VaR_3_4 = var(sim_returns[['Price3','Price4']].dot(newest_prices[2:]))
print("the VaR is {:.6f}, $VaR is {:.6f}.".format(VaR_3_4/sum(newest_prices[2:]),VaR_3_4))

the VaR is 703173.342301, $VaR is 140488344.093382.


## c. Calculate VaR (5%) for a portfolio of all 4 assets. (3pt)

In [186]:
VaRall = var(sim_returns.dot(newest_prices))
print("the VaR is {:.6f}, $VaR is {:.6f}.".format(VaRall/sum(newest_prices),VaRall))

the VaR is 905498.287507, $VaR is 372170401.436708.
